## Bayesian methods of hyperparameter optimization

In [1]:
#! pip install bayesian-optimization

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

#import warnings
#warnings.filterwarnings('ignore')
#import numpy as np
#import pandas as pd
#import lightgbm
#from bayes_opt import BayesianOptimization
#from catboost import CatBoostClassifier, cv, Pool
#from lightgbm import early_stopping, log_evaluation


In [21]:
!pip install catboost

In [3]:
import os
os.listdir()

['-2068008771550728719.PDF',
 '-2425312920492370960.PDF',
 '-3528592361582296881.PDF',
 '-446802187284581618.pdf',
 '-6040031645030520024.PDF',
 '-8412726975532546332.PDF',
 '.editorconfig',
 '.ipynb_checkpoints',
 '0185_T_PGO_Eleganza-4x3.potx',
 '03.10-Working-With-Strings.ipynb',
 '03_exploratory_data_analysis copy 2.ipynb',
 '1 in 13,983,816 (1).png',
 '1 in 13,983,816 (14).png',
 '1 in 13,983,816 (15).png',
 '1 in 13,983,816 (16).png',
 '1 in 13,983,816 (17).png',
 '1 in 13,983,816 (18).png',
 '1 in 13,983,816 (19).png',
 '1 in 13,983,816 (2).png',
 '1 in 13,983,816 (20).png',
 '1 in 13,983,816 (3).png',
 '1 in 13,983,816 (4).png',
 '1 in 13,983,816 (5).png',
 '1 in 13,983,816 (6).png',
 '1 in 13,983,816.png',
 '1-7618817781_Application_20190419_134245.pdf',
 '10000000_1222806788098714_7110332290584510144_n.mp4',
 '10000000_764873091003877_3941929143020424185_n.mp4',
 '1028714060868713200.PDF',
 '11.4.1 Case Study - Linear Regression',
 '11641-489299-0-2022-2023-03-11-04-29-13-708

In [4]:
def simple_func(a, b):
    return a + b

In [5]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

In [6]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 6.092     | 1.51      | 4.583     |
| 2         | 7.225     | 2.845     | 4.38      |
| 3         | 8.309     | 1.827     | 6.483     |
| 4         | 8.311     | 1.813     | 6.498     |
| 5         | 10.0      | 3.0       | 7.0       |


In [7]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

In [8]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

In [9]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [10]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [11]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering

In [12]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

In [13]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

In [14]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

In [15]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [16]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

In [20]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [3]:
#lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
#                                                'max_depth': (5, 63),
#                                                'lambda_l2': (0.0, 0.05),
#                                                'lambda_l1': (0.0, 0.05),
#                                                'min_child_samples': (50, 10000),
#                                                'min_data_in_leaf': (100, 2000)
#                                                })

#lgbBO.maximize(n_iter=10, init_points=2)

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [ ]:
lgbBO.max

Review the process at each step by using the '.res[0]' function.

In [ ]:
lgbBO.res[0]